# nbdbt

> Use jupyter notebooks for dbt development

[`dbt`](https://getdbt.com) (data build tool) is revolutionizing the way we do the process of extracting value from our data. 

This package allows Jupyter notebooks to be used for developing dbt models and analyses in complement with other dbt command line tools and VS Code extensions.

## Install

`pip install nbdbt`

## How to use

The `%%dbt` cell magic allows you to create models and analyses in your dbt project.

In addition, this package also leverages the `fal` package which provides useful modules for using dbt in python.

In [ ]:
# hide
# notest
%reload_ext autoreload
%reload_ext lab_black
%autoreload 2

In [ ]:
# load dbt cell magic
%reload_ext nbdbt.dbt_cellmagic

In [ ]:
# delete my_third_model (if existing)
! rm -f ../my_dbt_project/models/my_third_model.sql

This will create a new model `my_third_model` and compile it as well

In [ ]:
%%dbt -p ../my_dbt_project -a my_third_model -n notebooks/index.ipynb models/my_third_model.sql
select *
from {{ ref('my_second_dbt_model') }}

Leveraging the fal package, we assigned the result to the `sample1` variable

In [ ]:
my_third_model

The `ref` method allows us to run the query and save the results into a dataframe.

In [ ]:
results = my_third_model.ref()
results  # dataframe

,id
0,1.0


In [ ]:
print(my_third_model._compiled_sql)

-- AUTOGENERATED! DO NOT EDIT! File to edit: notebooks/index.ipynb (unless otherwise specified).
select *
from `sample-dbt-learn-project`.`jaffle_shop`.`my_second_dbt_model`
